In [23]:
from qiskit import IBMQ, QuantumRegister, ClassicalRegister, QuantumCircuit, BasicAer, execute
from qiskit_textbook.tools import array_to_latex
import numpy as np

In [24]:
def check_result(results, col, row):

    for i in results: 
        
        print("Quantum circuit solution")
        print(i+"\n")
        i=[int(char) for char in i]
        print("Reversing Array to get x bits first and changing 0's to -1's")
        i=np.array(list(reversed(i)))
        i=np.where(i==0,-1,i)
        # bring array in correct i.e. from incoming y2,y1,x2,x1 -> x1,x2,y1,y2
        a = i[0:2]
        
        a_prod = np.prod(a)
        a3 = 1*int(a_prod)
        a = np.append(a,a3)
        winning_alice = int(np.prod(a)) == 1
        # compute x3 by x1*x2

        b = i[2:]
        b_prod = np.prod(b)
        b3 = -1*int(b_prod)
        b = np.append(b,b3)
        winning_bob = int(np.prod(b)) == -1
        # compute y3 by negating y1*y2

        # compute overall winning condition -> alice at index of bob's row and vice versa
        
        print("Alice's selection. Column:"+str(col)+"\n"+str(a.reshape(3,1))+"\n")
        print("Bob's circuit solution. Row:"+str(row)+"\n"+str(b)+"\n")
        
        if(winning_bob & winning_alice & (a[row-1]==b[col-1])):
            print("Victory!")    
            print("The following magic square wins the game!")
            k = np.empty((3,3,))
            k[:] = np.nan
            k[:,col-1] = a
            k[row-1,:] = b
            print(k)
        else:
            print("The Game is lost!")
        print("------------------------------------------")  
        print("\n")

def makeEntangleCircuit():
    qr=QuantumRegister(4,name="qr")
    cr=ClassicalRegister(4,name="cr")
    EPRpair = QuantumCircuit(qr, cr, name="entangle")
#     print(qr[0])
    for i in range(2):
        EPRpair.h(qr[i])
    
    EPRpair.cx(qr[0],qr[2])
    EPRpair.cx(qr[1],qr[3])
    return EPRpair
    
def Alice(qc,col):
    if col==1:
        qc.h(0)
        qc.i(1)
    elif col==2:
        qc.swap(0,1)
        qc.h(0)
        qc.i(1)
    elif col==3:
        qc.cx(0,1)
        qc.h(0)
        qc.i(1)
        
def Bob(qc,row):
    if row==1:
        qc.h(2)
        qc.h(3)
    if row==2:
        qc.swap(2,3)
    elif row==3:
        qc.z(2)
        qc.z(3)
        qc.cz(2,3)
        qc.h(2)
        qc.h(3)
        
print("Alice choice (column):")
alice = int(input())
print("Bob choice (row):")
bob = int(input())

#Draw the rest of the circuit based on Alice and Bob's selection
circuit=makeEntangleCircuit()
if 4>alice>0 and 4>bob>0:
    Alice(circuit,alice) 
    Bob(circuit,bob)
#     vect=sv(circuit)
#     array_to_latex(vect, pretext="\\text{Statevector} = ")
    circuit.barrier()
    circuit.measure(0,0)
    circuit.measure(1,1)
    circuit.measure(2,2)
    circuit.measure(3,3)
    
    circuit.draw(output='mpl')
    print(circuit)
    
    backend = BasicAer.get_backend('qasm_simulator') # define the backend
    job = execute(circuit, backend, shots=8192) # run the job simulation
    results = job.result().get_counts()
    #print(job.result().get_unitary(circuit))
    check_result(results, alice, bob)
    print(results)

Alice choice (column):
1
Bob choice (row):
1
      ┌───┐     ┌───┐      ░ ┌─┐         
qr_0: ┤ H ├──■──┤ H ├──────░─┤M├─────────
      ├───┤  │  └───┘┌───┐ ░ └╥┘┌─┐      
qr_1: ┤ H ├──┼────■──┤ I ├─░──╫─┤M├──────
      └───┘┌─┴─┐  │  ├───┤ ░  ║ └╥┘┌─┐   
qr_2: ─────┤ X ├──┼──┤ H ├─░──╫──╫─┤M├───
           └───┘┌─┴─┐├───┤ ░  ║  ║ └╥┘┌─┐
qr_3: ──────────┤ X ├┤ H ├─░──╫──╫──╫─┤M├
                └───┘└───┘ ░  ║  ║  ║ └╥┘
cr: 4/════════════════════════╩══╩══╩══╩═
                              0  1  2  3 
Quantum circuit solution
0000

Reversing Array to get x bits first and changing 0's to -1's
Alice's selection. Column:1
[[-1]
 [-1]
 [ 1]]

Bob's circuit solution. Row:1
[-1 -1 -1]

Victory!
The following magic square wins the game!
[[-1. -1. -1.]
 [-1. nan nan]
 [ 1. nan nan]]
------------------------------------------


Quantum circuit solution
1010

Reversing Array to get x bits first and changing 0's to -1's
Alice's selection. Column:1
[[-1]
 [ 1]
 [-1]]

Bob's circuit solution. Row:1

In [4]:
def sv(qc): #This was only used for testing purposes
    svs = BasicAer.get_backend("statevector_simulator")
    job = execute(qc, svs)
    ket = job.result().get_statevector()
#   print(ket)
    return ket